In [21]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [22]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/VI_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,6011217110,VI,"9010 Estate Cottage, Suite 2, CHRISTIANSTED, VI",17.746640,-64.703198,7.801097e+14
1,2616077208,VI,"4611 Tutu Park #200, ST THOMAS, VI",18.338310,-64.890935,7.803096e+14
2,8157738307,VI,"5409 Reef Condos, Christiansted, VI",17.746640,-64.703198,7.801097e+14
3,7079827106,VI,"5007 ESTATE SHOYS PLOT 7, CHRISTIANSTED, VI",17.746640,-64.703198,7.801097e+14
4,4157848405,VI,"4028 Annas Retreat, St Thomas, VI",18.340437,-64.891130,7.803096e+14


In [23]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,6011217110,VI,"9010 Estate Cottage, Suite 2, CHRISTIANSTED, VI",17.746640,-64.703198,7.801097e+14
1,2616077208,VI,"4611 Tutu Park #200, ST THOMAS, VI",18.338310,-64.890935,7.803096e+14
2,8157738307,VI,"5409 Reef Condos, Christiansted, VI",17.746640,-64.703198,7.801097e+14
3,7079827106,VI,"5007 ESTATE SHOYS PLOT 7, CHRISTIANSTED, VI",17.746640,-64.703198,7.801097e+14
4,4157848405,VI,"4028 Annas Retreat, St Thomas, VI",18.340437,-64.891130,7.803096e+14


In [24]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
195,2087237800,VI,"4ABC-1, HULL BAY, VI",18.365533,-64.949822,NaN


<h4>FIPS Script

In [25]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [26]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('78')] #GU 66/ VI 78
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/1 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [27]:
geo_id_list =geo_id 

Length Check

In [28]:
print(len(geo_id_list))

1


Matching FIPS to DataFrame

In [29]:
fips_fill = fips_check_scraped_null

In [30]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_15716/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
195,2087237800,VI,"4ABC-1, HULL BAY, VI",18.365533,-64.949822,[780309605005009]


Remove brackets

In [31]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_15716/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
195,2087237800,VI,"4ABC-1, HULL BAY, VI",18.365533,-64.949822,780309605005009


In [32]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z


Parse for Merge

In [33]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
195,2087237800,780309605005009


Read in base data

In [34]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/VI_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,916521,6011217110,VI,"9010 Estate Cottage, Suite 2, CHRISTIANSTED, VI",17.746640,-64.703198,7.801097e+14
1,916522,2616077208,VI,"4611 Tutu Park #200, ST THOMAS, VI",18.338310,-64.890935,7.803096e+14
2,916523,8157738307,VI,"5409 Reef Condos, Christiansted, VI",17.746640,-64.703198,7.801097e+14
3,916524,7079827106,VI,"5007 ESTATE SHOYS PLOT 7, CHRISTIANSTED, VI",17.746640,-64.703198,7.801097e+14
4,916525,4157848405,VI,"4028 Annas Retreat, St Thomas, VI",18.340437,-64.891130,7.803096e+14


Merge with MainFrame

In [35]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,916521,6011217110,VI,"9010 Estate Cottage, Suite 2, CHRISTIANSTED, VI",17.746640,-64.703198,7.801097e+14,NaN
1,916522,2616077208,VI,"4611 Tutu Park #200, ST THOMAS, VI",18.338310,-64.890935,7.803096e+14,NaN
2,916523,8157738307,VI,"5409 Reef Condos, Christiansted, VI",17.746640,-64.703198,7.801097e+14,NaN
3,916524,7079827106,VI,"5007 ESTATE SHOYS PLOT 7, CHRISTIANSTED, VI",17.746640,-64.703198,7.801097e+14,NaN
4,916525,4157848405,VI,"4028 Annas Retreat, St Thomas, VI",18.340437,-64.891130,7.803096e+14,NaN
...,...,...,...,...,...,...,...,...
285,916806,9409698408,VI,"1201 King Cross St, Christiansted, VI",17.744646,-64.704127,7.801097e+14,NaN
286,916807,8643378606,VI,HENRY E ROHLSEN AIRPORT CARGO BLDG KINGSHILL V...,17.699498,-64.797470,7.801097e+14,NaN
287,916808,4155348001,VI,"6100 RED HOOK QTRS, #2, STE. C3-1, ST. THOMAS, VI",18.325982,-64.851845,7.803096e+14,NaN
288,916809,4251498806,VI,"Island Medical Center Suite 6, Christiansted, VI",17.746640,-64.703198,7.801097e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [36]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,916521,6011217110,VI,"9010 Estate Cottage, Suite 2, CHRISTIANSTED, VI",17.746640,-64.703198,7.801097e+14,NaN,780109702002030.0
1,916522,2616077208,VI,"4611 Tutu Park #200, ST THOMAS, VI",18.338310,-64.890935,7.803096e+14,NaN,780309602003011.0
2,916523,8157738307,VI,"5409 Reef Condos, Christiansted, VI",17.746640,-64.703198,7.801097e+14,NaN,780109702002030.0
3,916524,7079827106,VI,"5007 ESTATE SHOYS PLOT 7, CHRISTIANSTED, VI",17.746640,-64.703198,7.801097e+14,NaN,780109702002030.0
4,916525,4157848405,VI,"4028 Annas Retreat, St Thomas, VI",18.340437,-64.891130,7.803096e+14,NaN,780309602003011.0


Drop Extra FIPS Columns

In [37]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,6011217110,VI,"9010 Estate Cottage, Suite 2, CHRISTIANSTED, VI",17.746640,-64.703198,780109702002030.0
1,2616077208,VI,"4611 Tutu Park #200, ST THOMAS, VI",18.338310,-64.890935,780309602003011.0
2,8157738307,VI,"5409 Reef Condos, Christiansted, VI",17.746640,-64.703198,780109702002030.0
3,7079827106,VI,"5007 ESTATE SHOYS PLOT 7, CHRISTIANSTED, VI",17.746640,-64.703198,780109702002030.0
4,4157848405,VI,"4028 Annas Retreat, St Thomas, VI",18.340437,-64.891130,780309602003011.0
...,...,...,...,...,...,...
285,9409698408,VI,"1201 King Cross St, Christiansted, VI",17.744646,-64.704127,780109702002042.0
286,8643378606,VI,HENRY E ROHLSEN AIRPORT CARGO BLDG KINGSHILL V...,17.699498,-64.797470,780109714002011.0
287,4155348001,VI,"6100 RED HOOK QTRS, #2, STE. C3-1, ST. THOMAS, VI",18.325982,-64.851845,780309601001015.0
288,4251498806,VI,"Island Medical Center Suite 6, Christiansted, VI",17.746640,-64.703198,780109702002030.0


In [38]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z


Convert to File

In [39]:
fips_VI = fips_merge_drop

In [40]:
fips_VI.to_csv("../../../data/state_data/geo/geo_fips/23/VI_fips_scraped.csv") 